In [ ]:
# ---------------------------
# Cell 1: Install & Imports
# ---------------------------
!pip install tensorflow matplotlib pandas scikit-learn pillow --quiet

import os, zipfile, shutil, random
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from PIL import Image

# Optional mixed precision
try:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
except:
    pass


In [ ]:
# ---------------------------
# Cell 2: Mount Drive & unzip dataset
# ---------------------------
from google.colab import drive, files
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Paths
DRIVE_ZIP_PATH = '/content/drive/MyDrive/DataSetPlantDiseases.zip'
EXTRACTED_ROOT = '/content/plant_disease_data'

# Extract ZIP if not already done
if not os.path.exists(EXTRACTED_ROOT):
    os.makedirs(EXTRACTED_ROOT, exist_ok=True)
    with zipfile.ZipFile(DRIVE_ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACTED_ROOT)
    print(f"✅ Dataset extracted to {EXTRACTED_ROOT}")
else:
    print(f"✅ Dataset already exists at {EXTRACTED_ROOT}")


Mounted at /content/drive
✅ Dataset extracted to /content/plant_disease_data


In [ ]:
# ---------------------------
# Cell 2a: Clean & smaller dataset creation
# ---------------------------
import random, shutil

SMALL_ROOT = '/content/plant_disease_data_small'
MAX_IMAGES_PER_CLASS = 100  # max images per class

# Step 1: Remove hidden files
for root, dirs, files_in_dir in os.walk(EXTRACTED_ROOT):
    for f in files_in_dir:
        if f.startswith('.') or f.lower() == 'thumbs.db':
            os.remove(os.path.join(root, f))

# Step 2: Detect dataset folder structure
ROOT_DIR = EXTRACTED_ROOT
for sub in os.listdir(EXTRACTED_ROOT):
    path = os.path.join(EXTRACTED_ROOT, sub)
    if os.path.isdir(path):
        subfolders = os.listdir(path)
        if all(f in subfolders for f in ['train', 'valid', 'test']):
            ROOT_DIR = path
            print(f"✅ Dataset folder found: {ROOT_DIR}")
            break

# Step 3: Create smaller dataset
folders = ['train', 'valid', 'test']
for folder in folders:
    src_folder = os.path.join(ROOT_DIR, folder)
    dst_folder = os.path.join(SMALL_ROOT, folder)
    os.makedirs(dst_folder, exist_ok=True)

    class_dirs = [d for d in os.listdir(src_folder) if os.path.isdir(os.path.join(src_folder, d))]

    for class_name in class_dirs:
        class_src = os.path.join(src_folder, class_name)
        class_dst = os.path.join(dst_folder, class_name)
        os.makedirs(class_dst, exist_ok=True)

        all_images = [f for f in os.listdir(class_src) if os.path.isfile(os.path.join(class_src, f))]
        selected_images = random.sample(all_images, min(MAX_IMAGES_PER_CLASS, len(all_images)))

        for img in selected_images:
            shutil.copy(os.path.join(class_src, img), os.path.join(class_dst, img))

print(f"✅ Smaller dataset created at {SMALL_ROOT}")


✅ Dataset folder found: /content/plant_disease_data/DataSetPlantDiseases
✅ Smaller dataset created at /content/plant_disease_data_small


In [ ]:
# ---------------------------
# Cell 3: Train/Valid/Test directories
# ---------------------------

ROOT_DIR = SMALL_ROOT

train_dir = os.path.join(ROOT_DIR, 'train')
val_dir   = os.path.join(ROOT_DIR, 'valid')
test_dir  = os.path.join(ROOT_DIR, 'test')

# Verify existence
for folder in [train_dir, val_dir, test_dir]:
    if not os.path.exists(folder):
        raise FileNotFoundError(f"Folder not found: {folder}")
    else:
        print(f"✅ Found folder: {folder}")


✅ Found folder: /content/plant_disease_data_small/train
✅ Found folder: /content/plant_disease_data_small/valid
✅ Found folder: /content/plant_disease_data_small/test


In [ ]:
# ---------------------------
# Cell 4: Data Generators
# ---------------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123

# Training generator with strong augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation / Test generator
val_datagen = ImageDataGenerator(rescale=1./255)

# Generators
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED
)

test_gen = val_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED
)

# Dynamically detect number of classes
NUM_CLASSES = len(train_gen.class_indices)
print(f"✅ Number of classes detected: {NUM_CLASSES}")


Found 3800 images belonging to 38 classes.
Found 3800 images belonging to 38 classes.
Found 24 images belonging to 8 classes.
✅ Number of classes detected: 38


In [ ]:
# ---------------------------
# Cell 5: Model Compilation & Evaluation
# ---------------------------

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras import optimizers

def compile_and_summary(model, lr=1e-3):
    model.compile(
        optimizer=optimizers.Adam(lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    model.summary()

def evaluate_model_on_generator(model, generator):
    steps = int(np.ceil(generator.samples / generator.batch_size))
    preds = model.predict(generator, steps=steps, verbose=1)

    y_pred = np.argmax(preds, axis=1)
    y_true = generator.classes[:len(y_pred)]
    labels = list(generator.class_indices.keys())

    report = classification_report(y_true, y_pred, target_names=labels, zero_division=0, output_dict=True)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    return {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'report': report,
        'y_true': y_true,
        'y_pred': y_pred,
        'labels': labels
    }

print("✅ Utility functions ready")


✅ Utility functions ready


In [ ]:
# ---------------------------
# Cell 6: Model A Training (MobileNetV2) + Save to Google Drive (Timestamped)
# ---------------------------

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os, shutil, datetime

# Base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],3))
base_model.trainable = False

# Custom head
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)

model_a = models.Model(inputs=base_model.input, outputs=outputs)
compile_and_summary(model_a, lr=1e-3)

# Create timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# File paths
final_model_drive = f"/content/drive/MyDrive/model_a_final_{timestamp}.h5"
best_model_colab = f"/content/model_a_best_{timestamp}.h5"
best_model_drive = f"/content/drive/MyDrive/model_a_best_{timestamp}.h5"

# Callbacks
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss'),
    ReduceLROnPlateau(patience=2, factor=0.5, monitor='val_loss', verbose=1),
    ModelCheckpoint(best_model_colab, save_best_only=True, monitor='val_loss')
]

# 🚨 Run training only once
if 'history_a' not in globals():
    print("▶️ Starting training for 5 epochs...")
    history_a = model_a.fit(
        train_gen,
        epochs=5,
        validation_data=val_gen,
        callbacks=callbacks,
        verbose=1
    )
    print("✅ Training finished")

    # Save final model
    model_a.save(final_model_drive)
    print(f"💾 Final model saved to {final_model_drive}")

    # Save best checkpoint
    shutil.copy(best_model_colab, best_model_drive)
    print(f"🏆 Best checkpoint saved to {best_model_drive}")

else:
    print("⚠️ Training already done. Skip re-run.")


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,595,686 (9.90 MB)

 Trainable params: 337,702 (1.29 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

⚠️ Training already done. Skip re-run.


In [ ]:
# ---------------------------
# Cell 7: Save existing trained model to Google Drive
# ---------------------------

from google.colab import drive
import shutil
import os
import datetime

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Create timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# 3️⃣ Source files in Colab (from last training run)
colab_final_model = "/content/model_a_final.h5"   # change if your model has a different name
colab_best_model  = "/content/model_a_best.h5"

# 4️⃣ Destination paths in Drive
drive_final_model = f"/content/drive/MyDrive/model_a_final_{timestamp}.h5"
drive_best_model  = f"/content/drive/MyDrive/model_a_best_{timestamp}.h5"

# 5️⃣ Copy files to Drive
if os.path.exists(colab_final_model):
    shutil.copy(colab_final_model, drive_final_model)
    print(f"✅ Final model saved to Drive: {drive_final_model}")
else:
    print("⚠️ Final model not found in Colab.")

if os.path.exists(colab_best_model):
    shutil.copy(colab_best_model, drive_best_model)
    print(f"🏆 Best checkpoint saved to Drive: {drive_best_model}")
else:
    print("⚠️ Best checkpoint not found in Colab.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⚠️ Final model not found in Colab.
🏆 Best checkpoint saved to Drive: /content/drive/MyDrive/model_a_best_20250930_045325.h5


In [ ]:
# Save the final trained model manually
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_drive = f"/content/drive/MyDrive/model_a_final_{timestamp}.h5"

model_a.save(final_model_drive)
print(f"💾 Final model saved to Drive: {final_model_drive}")


💾 Final model saved to Drive: /content/drive/MyDrive/model_a_final_20250930_045614.h5


In [ ]:
# Resume training for additional epochs
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import datetime, shutil, os

# Current model is already loaded in `model_a`
# Number of additional epochs
additional_epochs = 3

# Create timestamp for new files
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# File paths for saving
best_model_colab = f"/content/model_a_best_{timestamp}.h5"
final_model_drive = f"/content/drive/MyDrive/model_a_final_{timestamp}.h5"

# Callbacks
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss'),
    ReduceLROnPlateau(patience=2, factor=0.5, monitor='val_loss', verbose=1),
    ModelCheckpoint(best_model_colab, save_best_only=True, monitor='val_loss')
]

# Resume training
history_resume = model_a.fit(
    train_gen,
    epochs=additional_epochs,
    validation_data=val_gen,
    callbacks=callbacks
)

# Save updated models to Drive
model_a.save(final_model_drive)
shutil.copy(best_model_colab, f"/content/drive/MyDrive/{os.path.basename(best_model_colab)}")

print(f"✅ Resumed training completed. Final model saved as: {final_model_drive}")
print(f"🏆 Best checkpoint saved as: /content/drive/MyDrive/{os.path.basename(best_model_colab)}")


Epoch 1/3
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.2348 - loss: 2.9583

119/119 ━━━━━━━━━━━━━━━━━━━━ 1650s 14s/step - accuracy: 0.2360 - loss: 2.9524 - val_accuracy: 0.7139 - val_loss: 1.0939 - learning_rate: 0.0010
Epoch 2/3
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.6401 - loss: 1.2334

119/119 ━━━━━━━━━━━━━━━━━━━━ 1636s 14s/step - accuracy: 0.6402 - loss: 1.2327 - val_accuracy: 0.8005 - val_loss: 0.7408 - learning_rate: 0.0010
Epoch 3/3
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.7160 - loss: 0.9138

119/119 ━━━━━━━━━━━━━━━━━━━━ 1634s 14s/step - accuracy: 0.7161 - loss: 0.9132 - val_accuracy: 0.8092 - val_loss: 0.6506 - learning_rate: 0.0010


✅ Resumed training completed. Final model saved as: /content/drive/MyDrive/model_a_final_20250930_050137.h5
🏆 Best checkpoint saved as: /content/drive/MyDrive/model_a_best_20250930_050137.h5


In [ ]:
# ---------------------------
# Cell 8: Ultra-Lightweight CNN + Fast Training + Auto Save to Drive
# ---------------------------

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from google.colab import drive
import os, datetime, pickle

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Build ultra-light CNN
inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

x = layers.Conv2D(16, (3,3), activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D((2,2))(x)

x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2))(x)

x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)

model_b = models.Model(inputs, outputs)

# 3️⃣ Compile
compile_and_summary(model_b, lr=0.002)

# 4️⃣ Callbacks with Drive saving
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
SAVE_DIR = '/content/drive/MyDrive'
best_model_file = os.path.join(SAVE_DIR, f'model_b_best_{timestamp}.h5')
final_model_file = os.path.join(SAVE_DIR, f'model_b_final_{timestamp}.h5')
history_file = os.path.join(SAVE_DIR, f'history_b_{timestamp}.pkl')

callbacks = [
    EarlyStopping(patience=2, restore_best_weights=True, monitor='val_loss'),
    ReduceLROnPlateau(patience=1, factor=0.5, monitor='val_loss', verbose=1),
    ModelCheckpoint(best_model_file, save_best_only=True, monitor='val_loss')
]

# 5️⃣ Train
history_b = model_b.fit(
    train_gen,
    epochs=5,      # ultra-short for speed
    validation_data=val_gen,
    batch_size=128,  # larger batch to reduce steps
    callbacks=callbacks
)

# 6️⃣ Save final model & history
model_b.save(final_model_file)
with open(history_file, 'wb') as f:
    pickle.dump(history_b.history, f)

print(f"✅ Model B final saved: {final_model_file}")
print(f"🏆 Model B best checkpoint saved: {best_model_file}")
print(f"📊 Training history saved: {history_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 128, 128, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 64, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_10     │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 38)             │         2,470 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,214 (118.02 KB)

 Trainable params: 30,214 (118.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.0325 - loss: 3.6265

238/238 ━━━━━━━━━━━━━━━━━━━━ 1925s 8s/step - accuracy: 0.0325 - loss: 3.6264 - val_accuracy: 0.0479 - val_loss: 3.5022 - learning_rate: 0.0020
Epoch 2/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.0576 - loss: 3.4789

238/238 ━━━━━━━━━━━━━━━━━━━━ 1893s 8s/step - accuracy: 0.0577 - loss: 3.4786 - val_accuracy: 0.1066 - val_loss: 3.2156 - learning_rate: 0.0020
Epoch 3/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.0950 - loss: 3.2412

238/238 ━━━━━━━━━━━━━━━━━━━━ 1931s 8s/step - accuracy: 0.0950 - loss: 3.2411 - val_accuracy: 0.1242 - val_loss: 3.1842 - learning_rate: 0.0020
Epoch 4/5
217/238 ━━━━━━━━━━━━━━━━━━━━ 2:42 8s/step - accuracy: 0.1190 - loss: 3.1213